<a href="https://colab.research.google.com/github/MohuaSinha/EVA4/blob/master/S12/Assignment%20A/S12_TinyImageNet_ResNet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd "//content/gdrive/My Drive/Colab Notebooks/EVA4/S12"
import sys
workingdir = '/content/gdrive/My Drive/Colab Notebooks/EVA4/S12'
sys.path.append(workingdir)

!unzip -q "/content/gdrive/My Drive/Colab Notebooks/EVA4/S12/tiny-imagenet-200.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls-ltr

/content/gdrive/My Drive/Colab Notebooks/EVA4/S12
total 242290
drwx------ 5 root root      4096 Feb  9  2015 tiny-imagenet-200/
-rw------- 1 root root 248100043 Apr 15 14:18 tiny-imagenet-200.zip


# **Extract test and training data from Tiny ImageNet DataSet**

In [5]:
import torch
import time
import scipy.ndimage as nd
import numpy as np
import imageio

path = 'tiny-imagenet-200/'

def get_id_dictionary():
    id_dict = {}
    for i, line in enumerate(open( path + 'wnids.txt', 'r')):
        id_dict[line.replace('\n', '')] = i
    return id_dict
  
def get_class_to_id_dict():
    id_dict = get_id_dictionary()
    all_classes = {}
    result = {}
    for i, line in enumerate(open( path + 'words.txt', 'r')):
        n_id, word = line.split('\t')[:2]
        all_classes[n_id] = word
    for key, value in id_dict.items():
        result[value] = (key, all_classes[key])      
    return result

def get_data(id_dict):
    print('starting loading data')
    train_data, test_data = [], []
    train_labels, test_labels = [], []
    t = time.time()
    for key, value in id_dict.items():
        train_data += [imageio.imread( path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i)),pilmode='RGB') for i in range(500)]
        train_labels_ = np.array([[0]*200]*500)
        train_labels_[:, value] = 1
        train_labels += train_labels_.tolist()

    for line in open( path + 'val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        test_data.append(imageio.imread( path + 'val/images/{}'.format(img_name) ,pilmode='RGB'))
        test_labels_ = np.array([[0]*200])
        test_labels_[0, id_dict[class_id]] = 1
        test_labels += test_labels_.tolist()

    print('finished loading data, in {} seconds'.format(time.time() - t))
    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)
  
train_data, train_labels, test_data, test_labels = get_data(get_id_dictionary())

print( "train data shape: ",  train_data.shape )
print( "train label shape: ", train_labels.shape )
print( "test data shape: ",   test_data.shape )
print( "test_labels.shape: ", test_labels.shape )

starting loading data
finished loading data, in 237.36653304100037 seconds
train data shape:  (100000, 64, 64, 3)
train label shape:  (100000, 200)
test data shape:  (10000, 64, 64, 3)
test_labels.shape:  (10000, 200)


# **Shuffle training data**

In [40]:
from torch.utils.data.sampler import SubsetRandomSampler


def shuffle_data(train_data, train_labels):
    num_train = len(train_data)
    indices = list(range(num_train))
    split=int(np.floor(0.3*num_train))

    np.random.shuffle(indices)
    train_idx,test_idx=indices[split:],indices[:split]

    train_sampler=SubsetRandomSampler(train_idx)
    test_sampler=SubsetRandomSampler(test_idx)

    normalize = transforms.Normalize(mean = [0.44785526394844055, 0.41693055629730225, 0.36942949891090393],
                                     std = [0.2928885519504547, 0.28230994939804077, 0.2889912724494934])
    train_dataset = datasets.ImageFolder('tiny-imagenet-200/train',
        transforms.Compose([
            transforms.RandomCrop(64, padding = 4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))

    #train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 1, shuffle = True, num_workers = 4, pin_memory = False)

    DataLoader(train_dataset, batch_size = 1, shuffle = False, num_workers = 0, pin_memory = False)
    trainloader= torch.utils.data.DataLoader(train_data,sampler=train_sampler,batch_size=64)
    testloader=torch.utils.data.DataLoader(train_data,sampler=test_sampler,batch_size=64)

    #return train_data[train_idx], train_labels[train_idx],test_data[test_idx], test_labels[test_idx]
    return trainloader,testloader
  
trainloader,testloader = shuffle_data(train_data, train_labels)
print(len(train_data))
print(train_data.shape)
print(testloader.size)

100000
(100000, 64, 64, 3)


AttributeError: ignored